In [20]:
import sys
sys.path.append("../")

import sympy as sp
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import backend_source_dir
from affine_body_core import compute_J_point, compute_J_vec

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")

ref: [Affine Body Prismatic Joint](https://spirimirror.github.io/libuipc-doc/specification/constitutions/affine_body_prismatic_joint/)

![Affine Body Prismatic Joint](../../docs/specification/constitutions/media/affine_body_prismatic_joint_fig1.drawio.svg)

$
\begin{aligned}
C_0 &= (\mathbf{c}_j - \mathbf{c}_i) \times \hat{\mathbf{t}}_i &= \mathbf{0} \\
C_1 &= (\mathbf{c}_i - \mathbf{c}_j) \times \hat{\mathbf{t}}_j &= \mathbf{0} \\
C_2 &= \hat{\mathbf{n}}_i - \hat{\mathbf{n}}_j &= 0 \\
C_3 &= \hat{\mathbf{b}}_i - \hat{\mathbf{b}}_j &= 0
\end{aligned}
$$

In [21]:
kappa  = Eigen.Scalar("kappa")

ci_bar = Eigen.Vector("ci_bar",3)
ti_bar = Eigen.Vector("ti_bar",3)
ni_bar = Eigen.Vector("ni_bar",3)
bi_bar = Eigen.Vector("bi_bar",3)

cj_bar = Eigen.Vector("cj_bar",3)
tj_bar = Eigen.Vector("tj_bar",3)
nj_bar = Eigen.Vector("nj_bar",3)
bj_bar = Eigen.Vector("bj_bar",3)

$$
C_0 = (\mathbf{c}_j - \mathbf{c}_i) \times \hat{\mathbf{t}}_i = \mathbf{0} 
$$

$$
\mathbf{F}_0 = \begin{bmatrix}
\mathbf{c}_j - \mathbf{c}_i \\
\hat{\mathbf{t}}_i \\
\end{bmatrix}_{6 \times 1}
$$

Frame Affine Body Mapping:

$$
\mathbf{J}_0 = \begin{bmatrix}
-\mathbf{J}(\bar{\mathbf{c}}_i) & \mathbf{J}(\bar{\mathbf{c}}_j) \\
\mathring{\mathbf{J}}(\bar{\mathbf{t}}_i) & \\
\end{bmatrix}_{6 \times 24}
$$

$$
\mathbf{F}_0 = \mathbf{J}_0 
\begin{bmatrix}
\mathbf{q}_i \\
\mathbf{q}_j \\
\end{bmatrix}
$$

In [ ]:
J0 = sp.Matrix.zeros(6,24)
J0[0:3, 0:12] = -compute_J_point(ci_bar)
J0[0:3, 12:24] = compute_J_point(cj_bar)
J0[3:6, 0:12] = -compute_J_vec(ti_bar)

G6 = Eigen.Vector("G6",6)
H6x6 = Eigen.Matrix("H6x6",6,6)

J0T_G0 = J0.T * G6
J0T_H0_J0 = J0.T * H6x6 * J0

F0_ = sp.Matrix.zeros(6,1)

F0 = Eigen.Vector("F0", 6)

dij = F0[0:3, 0]
ti = F0[3:6, 0]

C0 = dij.cross(ti)
E0 = kappa * C0.dot(C0)/2

Cl_E0 = Gen.Closure(F0)




dE0dF0 = VecDiff(E0, F0)
# display(dE0dF0)
ddE0ddF0 = VecDiff(dE0dF0, F0)
# display(ddE0ddF0)




$$
C_1 = (\mathbf{c}_i - \mathbf{c}_j) \times \hat{\mathbf{t}}_j = \mathbf{0} 
$$

$$
\mathbf{F}_1 = \begin{bmatrix}
\mathbf{c}_i - \mathbf{c}_j \\
\hat{\mathbf{t}}_j \\
\end{bmatrix}_{6 \times 1}
$$

Frame Affine Body Mapping:

$$
\frac{\partial \mathbf{F}_1}{\partial 
\begin{bmatrix}
\mathbf{q}_i \\
\mathbf{q}_j \\
\end{bmatrix}
} =
\mathbf{J}_1 = \begin{bmatrix}
\mathbf{J}(\bar{\mathbf{c}}_i) & -\mathbf{J}(\bar{\mathbf{c}}_j) \\
\mathring{\mathbf{J}}(\bar{\mathbf{t}}_j) & \\
\end{bmatrix}_{6 \times 24}
$$

In [ ]:
F1 = Eigen.Vector("F1", 6)
dji = F1[0:3, 0]
tj = F1[3:6, 0]

C1 = dji.cross(tj)
E1 = kappa * C1.dot(C1)/2

J1 = sp.Matrix.zeros(6,24)
J1[0:3, 0:12] = compute_J_point(ci_bar)



In [23]:
# s = f"""// Affine Body Prismatic Joint
# {ClE("dEdQ", dEdQ)}
# {ClE("ddEddQ", ddEddQ)}
# {ClG("GradE", Grad)}
# {ClH("HessE", Hess)}
# {ClX("ddEddX", ddEddX)}
# {ClXG("GradX", GradX)}
# {ClXH("HessX", HessX)}
# """
# print(s)

# f = open(
#     backend_source_dir("cuda")
#     / "affine_body/constitutions/sym/affine_body_prismatic_joint.inl",
#     "w",
# )
# f.write(s)
# f.close()
